In [52]:
# For all setpoint params 15
# Environment
# Agent
# Training
import pickle

![](NewRLAGC.png)

In [99]:
import numpy as np
class AGCRLEnv:
    def __init__(self,observations,actions,action_parameter,action_space):
        """initialise action space, observation space & load data"""
        self.action_parameter=action_parameter
        self.action_space=action_space
        self.observations=observations
        self.actions=actions
        self.index=0
        self.teamindex=0
        self.observation_space=self.observations[self.teamindex].iloc[0].shape
        self.curr_obs=self.observations[self.teamindex].iloc[self.index]
        self.next_obs=self.curr_obs=self.observations[self.teamindex].iloc[self.index+1]
        self.curr_reward=0
        self.ep_reward=0
        self.interval=action_space[1]-action_space[0]
        
    def estimate_closest_as(self,value):
        return self.action_space[int(value)//int(self.interval)]
    
    def step(self,action):
        """
        return reward and next obs 
        """
        self.reward=self.rewardfunc(action)
        self.ep_reward+=self.reward
        self.index+=1
        cur_obs=self.curr_obs
        next_obs=self.next_obs
        if self.index>=len(self.observations[self.teamindex]):
            self.reset()
            return self.curr_obs,self.reward,True 
        self.curr_obs=self.observations[self.teamindex].iloc[self.index]
        self.next_obs=self.curr_obs=self.observations[self.teamindex].iloc[self.index+1]
        return self.curr_obs,self.reward,False
    
    def rewardfunc(self,action):
        """
        action with action at current index in action if equal positive or else negative
        """
#         print(self.action_space[action])
#         print(self.actions[self.teamindex]["assim_sp"][self.index])
        if(self.action_space[action]==self.estimate_closest_as(self.actions[self.teamindex]["assim_sp"][self.index])):
            return 100
        else:
            return -1*abs(self.action_space[action]-self.actions[self.teamindex]["assim_sp"][self.index])
    
    def reset(self):
        """
        set index to 0 and increment team index by 1 if greater than 4 go back to 0
        """
        self.teamindex+=1
        if self.teamindex>=5:
            self.teamindex=0
        self.index=0
        self.observation_space=self.observations[self.teamindex].iloc[0].shape
        self.curr_obs=self.observations[self.teamindex].iloc[self.index]
        self.next_obs=self.curr_obs=self.observations[self.teamindex].iloc[self.index+1]
        self.curr_reward=0
        self.ep_reward=0
    

In [100]:
with open('observations.pickle', 'rb') as handle:
    obs = pickle.load(handle)

In [101]:
with open('actions.pickle', 'rb') as handle:
    actions = pickle.load(handle)

In [102]:
env=AGCRLEnv(obs,actions,"assim_sp",np.linspace(0,100,21))

In [103]:
env.action_space,env.observation_space

(array([  0.,   5.,  10.,  15.,  20.,  25.,  30.,  35.,  40.,  45.,  50.,
         55.,  60.,  65.,  70.,  75.,  80.,  85.,  90.,  95., 100.]),
 (39,))

In [104]:
# env.index=10010

In [91]:
obs,reward,done=env.step(20)

In [92]:
reward,done

(100, False)

In [95]:
obs,reward,done=env.step(7)

In [96]:
reward

-65.0

In [97]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Activation, Flatten
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
from collections import deque
import tensorflow as tf
import numpy as np

import time
import random
from tqdm import tqdm
# Own Tensorboard class

DISCOUNT = 0.99
REPLAY_MEMORY_SIZE = 5000  # How many last steps to keep for model training
# Minimum number of steps in a memory to start training
MIN_REPLAY_MEMORY_SIZE = 1000
MINIBATCH_SIZE = 1000  # How many steps (samples) to use for training
UPDATE_TARGET_EVERY = 500  # Terminal states (end of episodes)
MODEL_NAME = 'RLAGC'
MIN_REWARD = -1000  # For model save
# MEMORY_FRACTION = 0.20


class ModifiedTensorBoard(TensorBoard):

    # Overriding init to set initial step and writer (we want one log file for all .fit() calls)
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.step = 1
        self.writer = tf.summary.create_file_writer(self.log_dir)
        self._log_write_dir = self.log_dir

    # Overriding this method to stop creating default log writer
    def set_model(self, model):
        pass

    # Overrided, saves logs with our step number
    # (otherwise every .fit() will start writing from 0th step)
    def on_epoch_end(self, epoch, logs=None):
        self.update_stats(**logs)

    # Overrided
    # We train for one batch only, no need to save anything at epoch end
    def on_batch_end(self, batch, logs=None):
        pass

    # Overrided, so won't close writer
    def on_train_end(self, _):
        pass

    # Custom method for saving own metrics
    # Creates writer, writes custom metrics and closes writer
    def update_stats(self, **stats):
        self._write_logs(stats, self.step)

    def _write_logs(self, logs, index):
        with self.writer.as_default():
            for name, value in logs.items():
                tf.summary.scalar(name, value, step=index)
                self.step += 1
                self.writer.flush()
# Agent class


class DQNAgent:
    def __init__(self, env, model=None):

        # Used to count when to update target network with main network's weights
        self.target_update_counter = 0
        self.env = env
        self.action_space = env.action_space

        if model == None:
            # Main model
            self.model = self.create_model()

            # Target network
            self.target_model = self.create_model()
            self.target_model.set_weights(self.model.get_weights())
        else:
            self.model = model
            self.target_model = self.create_model()
            self.target_model.set_weights(self.model.get_weights())

        # An array with last n steps for training
        self.replay_memory = deque(maxlen=REPLAY_MEMORY_SIZE)

        # Custom tensorboard object
        self.tensorboard = ModifiedTensorBoard(
            log_dir="logs/{}-{}".format(MODEL_NAME, int(time.time())))

    def create_model(self):
        model = Sequential()

        # OBSERVATION_SPACE_VALUES = (10, 10, 3) a 10x10 RGB image.
        model.add(Dense(16, input_shape=self.env.observation_space))
        model.add(Dense(32))
        model.add(Dense(16))
        # ACTION_SPACE_SIZE = how many choices (10)
        model.add(Dense(len(self.action_space), activation='linear'))
        model.compile(loss="mse", optimizer=Adam(
            lr=0.001), metrics=['accuracy'])
        return model

    # Adds step's data to a memory replay array
    # (observation space, action, reward, new observation space, done)
    def update_replay_memory(self, transition):
        self.replay_memory.append(transition)

    # Trains main network every step during episode
    def train(self, terminal_state, step):

        # Start training only if certain number of samples is already saved
        if len(self.replay_memory) < MIN_REPLAY_MEMORY_SIZE:
            return

        # Get a minibatch of random samples from memory replay table
        minibatch = random.sample(self.replay_memory, MINIBATCH_SIZE)
        # minibatch = self.replay_memory[-MINIBATCH_SIZE:]
        # Get current states from minibatch, then query NN model for Q values
        current_states = np.array([transition[0]
                                   for transition in minibatch])
        current_qs_list = self.model.predict(current_states)

        # Get future states from minibatch, then query NN model for Q values
        # When using target network, query it, otherwise main network should be queried
        new_current_states = np.array(
            [transition[3] for transition in minibatch])
        future_qs_list = self.target_model.predict(new_current_states)

        X = []
        y = []

        # Now we need to enumerate our batches
        for index, (current_state, action, reward, new_current_state, done) in enumerate(minibatch):

            # If not a terminal state, get new q from future states, otherwise set it to 0
            # almost like with Q Learning, but we use just part of equation here
            if not done:
                max_future_q = np.max(future_qs_list[index])
                new_q = reward + DISCOUNT * max_future_q
            else:
                new_q = reward

            # Update Q value for given state
            current_qs = current_qs_list[index]
            current_qs[action] = new_q

            # And append to our training data
            X.append(current_state)
            y.append(current_qs)

        # Fit on all samples as one batch, log only on terminal state
        self.model.fit(np.array(X), np.array(y), batch_size=MINIBATCH_SIZE, verbose=0,
                       shuffle=False, callbacks=[self.tensorboard] if terminal_state else None)

        # Update target network counter every episode
        if terminal_state:
            self.target_update_counter += 1

        # If counter reaches set value, update target network with weights of main network
        if self.target_update_counter > UPDATE_TARGET_EVERY:
            self.target_model.set_weights(self.model.get_weights())
            self.target_update_counter = 0

    # Queries main network for Q values given current observation space (environment state)
    def get_qs(self, state):
        return self.model.predict(np.array(state).reshape(1, self.env.observation_space[0]))[0]

In [98]:
len(env.observations[0])

47863